In [1]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


# 1. Build Toronto Postal Codes Dataframe

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### Extract Wikipedia table using Beautiful Soup

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#Build table object
My_table = soup.find('table',{'class':'wikitable sortable'})
whole_list=My_table.findAll("tr")
postalcode=[]
borough=[]
neighborhood=[]

### Loop to copy and paste each beautiful soup cell object into lists

In [4]:
i=0
for tr in My_table.findAll("tr"):
    sub_list=whole_list[i].findAll("td")
    if sub_list :
            postalcode.append(sub_list[0].get_text())
            borough.append(sub_list[1].get_text()) 
            spec_list=sub_list[2].string
            if spec_list!="Not assigned\n" :
                    spec_list=sub_list[2].findAll("a")
                    if spec_list :
                        neighborhood.append(spec_list[0].get('title'))
                    else:
                        neighborhood.append(sub_list[2].get_text().strip()) 
            else:
                    neighborhood.append(sub_list[2].get_text().strip())
    i=i+1

### Create and format dataframe from previous extracted data

In [5]:
#Create dataframe with data from previous 
df=pd.DataFrame()
df['PostalCode']=postalcode
df['Borough']=borough
df['Neighborhood']=neighborhood

#Remove from dataframe (Toronto) and ", Toronto"
df['Neighborhood']=df.loc[ : , 'Neighborhood' ].str.replace(r"\(.*\)","").str.replace(', Toronto','')

#Remove from dataframe "Not assigned" Borough rows
dt=df[df.Borough!= 'Not assigned']
dt = dt.reset_index(drop=True)

#Loop to copy Borough in "Not assigned" Neighborhood cells 
for idx, row in dt.iterrows():
    if dt['Neighborhood'].iloc[idx]=="Not assigned":
        dt.loc[idx,'Neighborhood']=dt.loc[idx,'Borough']

### Return Toronto Postal Codes Dataframe combining together Neighborhood with same postal code

In [6]:
dt = dt.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
dt.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Woburn
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
dt.shape

(103, 3)